In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np

In [2]:
!pwd
!ls

/home/yash/Desktop/work
Main.ipynb	   millionsongsubset_full.tar.gz  PythonSrc
MillionSongSubset  OS6-1.pdf


In [3]:
msd_subset_path = '/home/yash/Desktop/work/MillionSongSubset'
msd_subset_data_path = os.path.join(msd_subset_path, 'data')
msd_subset_addf_path = os.path.join(msd_subset_path, 'AdditionalFiles')
assert os.path.isdir(msd_subset_path), 'wrong path'

In [4]:
msd_code_path = '/home/yash/Desktop/work'
assert os.path.isdir(msd_code_path), 'wrong path'

sys.path.append(os.path.join(msd_code_path, 'PythonSrc'))
import hdf5_getters as GETTERS

In [5]:
# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

# we define this very useful function to iterate the files
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [6]:
print('number of song files:',apply_to_all_files(msd_subset_data_path))

number of song files: 10000


In [7]:
all_artist_names = set()
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print('all artist names extracted in:',strtimedelta(t1,t2))


all artist names extracted in: 0:01:22.711867


In [84]:
print('found',len(all_artist_names),'unique artist names')
for k in range(5):
    print(list(all_artist_names)[k].decode())

found 4412 unique artist names
The Kirkbys
Hawkshaw Hawkins
Stanley Clarke;Herbie Hancock
Corey / Shaquille O'Neal
Princess Lover


In [24]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))

In [25]:
# we build the SQL query
q = "SELECT DISTINCT artist_name FROM songs"
# we query the database
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print ('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
# we close the connection to the database
conn.close()

all artist names extracted (SQLite) in: 0:00:00.014942


In [53]:
import random
# let's see some of the content
x = random.randint(1, 10000)
for k in range(x, x + 10):
    try:
        print (all_artist_names_sqlite[k][0])
    except:
        print('Error')
        break

This Is Hell
This Moment
This Moment In Black History
This Or The Apocalypse
Thomas Battenstein
Thomas Dolby
Three 6 Mafia
Three-6 Mafia
Thrice
Thrilla U


In [71]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id FROM songs"
res = conn.execute(q)
all_artist_ids = list(map(lambda x: x[0], res.fetchall()))
conn.close()

In [79]:
for k in range(4):
    print(all_artist_ids[k])  

AR009211187B989185
AR00A6H1187FB5402A
AR00LNI1187FB444A5
AR00MBZ1187B9B5DB1


In [89]:
files_per_artist = {}
for aid in all_artist_ids:
    print(aid)
    files_per_artist[aid] = 0 

AR009211187B989185
AR00A6H1187FB5402A
AR00LNI1187FB444A5
AR00MBZ1187B9B5DB1
AR01IP11187B9AF5D2
AR01VU31187B997DA0
AR01W2D1187FB5912F
AR022JO1187B99587B
AR02DB61187B9A0B5E
AR02IU11187FB513F2
AR02KZG1187FB3B3B4
AR02PUA1187FB52574
AR02YGA1187B9B8AC4
AR035N21187FB3938E
AR039B11187B9B30D0
AR03BDP1187FB5B324
AR03GWG1187B9B6C85
AR040M31187B98CA41
AR040QX1187FB4CFE1
AR048JZ1187B9AEB85
AR04KTB1187B99B6B7
AR050VJ1187B9B13A7
AR051KA1187B98B2FF
AR0569B1187B9A4036
AR059HI1187B9A14D7
AR05IU31187B9B9A1A
AR05KQA1187B9963B3
AR05NQH1187B98E875
AR05OJD1187B9B99A6
AR05VW21187FB407B4
AR05XFO1187FB4DAE5
AR061J011A348F060F
AR0693R1187FB59D32
AR06EB01187FB40150
AR06EHB1187FB42914
AR06IFV1187B9977EC
AR06M3F1187FB38BED
AR06USW1187FB4EACA
AR07CRG1187FB594FA
AR07HWO1187B9A2189
AR07QYZ1187FB5BF78
AR07VS41187FB37944
AR08DYV1187B99A0D7
AR08GQF1187B98E316
AR08NXW1187B99526D
AR08YJF1187B9B2D59
AR0A30G1187B9B183B
AR0AVIB1187FB37D01
AR0B1CH1187FB3A824
AR0B3RS1187FB48F5D
AR0B5HI1187B9B9DB9
AR0B6VD1187B99C293
AR0B8CT1187B

ARSFWGS1269FB35F84
ARSG0RY1187B98E989
ARSG7NQ1187FB57482
ARSGZ3J1187FB3E610
ARSHP5P1187B999685
ARSHTBB1241B9C434A
ARSHZBZ1187B98A445
ARSIECN1257509EBB0
ARSJ46X1187FB4A62A
ARSK4D11187B9BA046
ARSKATF1187B98A622
ARSKDV01187B98A619
ARSKPIJ11F50C50608
ARSKSFC1264D0E32F8
ARSKZAQ1269FCD50EF
ARSL3N21187B98DFC5
ARSL5SP1187B9A7AE0
ARSLDFH12454A38A01
ARSM2YU1187B99D602
ARSMRIN1187B9AB35A
ARSMT991187B9A5C45
ARSNTX51187B989CA2
ARSNTZF12454A2DF53
ARSO8M21187B99C7EE
ARSOPEA1187B9B0F98
ARSPC9J1187B9A7EA1
ARSPIHU1187B99DDD8
ARSPQ0H1187B999944
ARSPUHE1187B99DB8A
ARSPUJF1187B9A14B8
ARSQDUN1187B98D7D7
ARSQFIK11F50C4D038
ARSQIM51187B9B460E
ARSQRPG11C8A4174C6
ARSQW3I1187B9ACD14
ARSRCI71187FB59188
ARSRFOY1187B9917DA
ARSS1OZ1187FB54440
ARSS4Y41187B9B0AC6
ARSSLTP1241B9CC51A
ARSSR8L1187B990BC5
ARSSW5A1187FB4B5E5
ARSTA431187B9A3599
ARSTH0R1187B98951E
ARSTSEH121318C5B01
ARSTXTP1187B99BD43
ARSVI4L1187B996181
ARSVRVA1187B99228A
ARSW5F51187FB4CFC9
ARSWORN1187B991A7B
ARSWXNX1187FB3D04E
ARSX3I61187B98FAC7
ARSX6IX1187B

In [94]:
def func_to_count_artist_id(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id.decode()] += 1
    h5.close()

In [98]:
apply_to_all_files(msd_subset_data_path,func=func_to_count_artist_id)

AR9YWMS1187FB43A34
ARQMYTM1187FB4678C
AR80RBU1187FB3757D
ARFNJ4O1187B9A204B
AR0S0EO1187FB490BE
AR3O66A1187FB39519
ARILV5E1187B9959C8
ARI945I1187FB4CC22
ARDK87R1187B98D51C
AR23EC41187FB4805D
ARQX3GF1187FB4F18B
ARPBTRI1187FB52457
ARPNY981187B9ACB75
ARBCUNV1187B98A9FB
ARUZLXZ1187FB3BFF1
ARGUNFT1187FB3B140
ARU9KWP1187B98B2A7
AR0XSOO1187B9B9E88
ARE36MM1187B991E50
ARC3JTL1187FB4A1CE
ARHINI31187B995C1D
AR5OB9M1187FB52F29
ARH0NCL1187B99B41A
ARBQ5TQ1187B992B8A
ARSCJQW1187B9B3523
AR0QS8F1187B9ADC96
ARQ0BG21187B99ED8F
AR7GOK91187FB455F5
ARBFDJW1187B9AD27A
ARS10A81187B9B6E16
ARTHEAP122BCFCD0B2
ARZ8S071187B993849
ARXOKNT1187B9AA521
ARY80281187FB3E380
ARR035B1187B99EF11
ARPISA41187FB3DE42
AR7GOK91187FB455F5
ARPEUYM1187FB4A97A
ARS2KW51187B9B556E
ARQHLIO1187B98AA6A
ARTUXSS1187FB4BD20
ARLLRL61187B9B6698
AR323JD1187FB483D0
ARX134S1187FB46E56
ARC8CQZ1187B98DECA
AR6HLKR1187B9A735A
ARJJPQP1241B9CA062
AR97EHA1187B98AD5F
ARVXKTV1187FB44139
ARNLHY61187FB3A0C0
ARRH2S81187B98C8AA
AR5ZI3K1187FB38BE0
ARHRY5S1187B

ARX8PZU1187B9AA7E2
AR3QRJ91187B98E4C2
ARGXJRQ1241B9C5FDE
ARO2PB81187B98EB4A
ARBKMHZ11F4C843491
AROC6N61187FB40B14
ARJXZX41187B9B6A43
ARQVLQJ1187B98F11F
AR7CCCC1187FB578FF
ARSG7NQ1187FB57482
AR342K51187B98B571
ARPKMIM1187FB397DE
AR82DJK1187B991107
AREB22L1187B9A4BCE
ARC43071187B990240
ARWUK9L11C8A42C37B
AREMOC71187B996C2F
ARKJKVO1187FB49D0A
ARWGLPE1187FB3E54C
ARSCJQW1187B9B3523
ARALP6I1187B989E27
ARFCOGY1187FB454B4
AR1DR0E1187FB48F61
ARTZMXB1187B990A55
ARXGWZP11F4C83C075
AR8WHEJ1187B98CDCA
ARC39PE1187B9B2C01
ARC8CQZ1187B98DECA
ARAI2ZO1187B9A0536
ARUGU0E1187B9B30F6
AR3AIEI1187B992D8D
ARYSHL01187B9AAEC7
AR5ZVJ91187B99083A
ARCZ20M1187FB49687
ARP7ILI1187B9AE9F7
ARKCYJV1187FB392E9
ARXB7B21187B98BEF0
AR6JVK31187FB58FE3
AR23MLO119B8668CDA
ARRWDU31187B9B5656
ARYKF861187B99C74B
ARF3GX71187FB3EB66
ARU9KWP1187B98B2A7
AR95CRV1187B9AFD6D
AR6KK741187B98C9E9
ARW071B1187B9A603C
AR2K2CM1187B98D4F1
AR08DYV1187B99A0D7
ARZLX2S1187B991A47
ARBR7II1187B9B821E
ARJDUON1187B989F69
ARGNGRI11E2835D567
ARQDWBG1187B

ARVAWTX1187B9A9990
ARD3ICK1187B9A56C9
ARCLCFS1187B9AE476
ARB06Z71187B99049A
AR63RWN1187FB3E89A
ARB0WNP1187B9AC4AB
ARJZQ321187FB5AAA1
ARCGJ6U1187FB4D01F
ARB78VW1187B98F81C
ARHG53Q1187B9ACBFC
ARGBR4I1187B99D073
ARWDDAE11F4C844B57
ARTSGR41187B9912C6
AR13CXU1187B9AD30A
ARQZRQ11187B9A5918
ARHB1961187B9B1732
ARZDYBC1187B9A3142
AR1CB5G1187B9AFB8E
ARO9XCZ1187B99C255
AR46CAD1187FB4D84B
ARBOG0H1187B99F775
AR6CLFP1187B9ACB94
ARRH63Y1187FB47783
AR2XH7R1187B998D30
ARL43EQ1187FB4D5EC
ARV21JS1187B99E516
AR81BPV1187B9AED0E
AR8CT8U1187FB4D803
ARMG34A1187FB548A8
ARVJSJY1250940FA2B
ARIVB351187B9B485E
ARMX5R91187B991E20
AR14CJ91187FB3A994
AR3ZV061187FB3C821
ARRT66G1187FB3DDF9
ARYDFJA1241B9CBFF2
ARN03F71187FB4E3F4
ARZFRQM1187B9A9772
ARJVTRE1187B9959C0
AR58SJE1187B9A8E4C
ARLOIDN1187B9B060D
ARIWQYF11F4C8427FF
ARF3XJD1187FB38DE0
ARYUHNV1187FB482AF
ARR2BHS1187B9B8FA7
AREVQ7D1187FB4371B
ARRMOXN122988EE92C
AR126G61187B9B88A6
ARLSF8H1187B9A76B0
ARNWZ1N1187B9B71BA
ARCF2WB1187FB54132
ARKSZW81187B9B695D
AR9G2A41187B

ARSYKGV1187B9A49CD
AR0X3JJ1187FB3A9A7
ARSZIBW1187B994053
AROVSYC119B3403921
AR1WU211187FB57E2B
ARAK3HK1187FB51B38
AROSW2I1187FB44866
AR8AZTM1187FB3FB9C
ARD4C1I1187FB4B0C3
ARU5CP31187FB44C3A
AR3752L1187FB4B67E
ARTMM411187FB56FD0
ARYOBYD1241B9C87F8
ARXZP9T1187FB4565D
ARZP2M21187FB59C63
AROMB601187B991317
ARQC4R31187B9A0D88
ARTWWNA1187B9B3BD7
ARBUOMY1187B99F2EB
ARQC4R31187B9A0D88
ARETCTY1187FB4C033
ARQSZLT124207825BA
ARUYGJ31187B9A2FEA
ARIJ7MR1187B9B484E
ARLNMEP11C8A41519E
ARGBR4I1187B99D073
AR4EVIX1187B993DE9
ARFNXFR1187B98F6C1
ARMRQFC1269FCD2C40
ARD1XMG1187FB41229
ARQOATR1187B9AE959
ARD7TVE1187B99BFB1
ARTP2PK1187B98FECA
AR51YM81187B9B536C
ARCZ20M1187FB49687
ARI3W3K1187FB49E6F
ARO0QD41187FB3AB0A
AR0WLH6119B8669027
ARHJJ771187FB5B581
AR1E4V11187B99F400
ARSYHZ21187B99F5F5
ARQI60P1187FB464C9
ARS8DXZ1187B998AA3
AREYIHC1187B99CE0B
ARY2ZUX1187B9A266C
AR05IU31187B9B9A1A
AR85QTQ11A348EFF96
AR5TQ1T1187FB59492
ARNVM131187B990D2B
ARMBDNQ1187FB3F3EC
AR76DOS1187FB5AE78
AR05OJD1187B9B99A6
ARD5SR21187B

ARPGYM11187FB4B1E1
ARHG53Q1187B9ACBFC
AR97V5X1187FB4AE98
ARRTI6E1187B9A661D
ARCLLH11187FB48DE0
AR0PUK71187B9AC3C0
ARFH6FT1187FB5BF47
AR6F6I21187FB5A3AA
ARELVCU1187B98F3E5
ARW70HR1187B9A358C
ARSL3N21187B98DFC5
ARSE6YI1187B9B9CD2
ARHNCQ31187B9915DC
AR0QT3O1187FB56860
ARMT23H1187B995083
AR6UQ8H1187B99DA9E
ARBGJLS11F4C83D4B9
ARJ45QE1187FB423B8
ARPDVPJ1187B9ADBE9
ART9Z1P1187FB36C94
ARDK0551187FB5AC48
ARHNMEZ11F50C4706C
ARNRPI31187B9AC699
ARAAP251187B99DDE0
ARDCIH31187B98B241
ARHP4FT1187FB3897C
ARSFWGS1269FB35F84
ARMRCET1187FB52049
AR8RXRI1187FB4104B
AROY6XW1187B991121
ARVML4B1187FB52324
ARJTX7M1187FB4CED3
ARK0BT61187B9ACEC2
ARRJFJ31187B9961EF
ARSFVDS1187B990F14
AR4SOKG1187B98D2B0
ARF3FRP1187B9A77BF
AR78ZID1187B9B31ED
ARZDYBC1187B9A3142
ARW45K91187FB3DE04
ARXWBOL11E2835E5BB
ARADCSF11F4C8416CD
ARTP2PK1187B98FECA
ARDW9RF1187B99002C
ARCDCFY122ECCBCC6A
ARTXJFJ1241B9C55EF
ARKC83D1187B9AB367
ARN03F71187FB4E3F4
AROP6331187FB398F7
ARNTRZQ122ECCBC53C
AR9OLFX1187B9A5762
ARB29H41187B98F0EF
ARQ4OZM1187B

...

ARQDONM1187B9BA1AD
ARPQ9MD1187FB3D29A
AR19QNR1187B9A13C5
ARNLO5S1187B9B80CC
ARIHI8H1187B9B0058
ARIK43K1187B9AE54C
ARHJMEF1187FB415AE
ARN3X4T1187B99EA39
ARPWMGF1269FB25878
AR93D4D1187FB3ACAD
ARNYDR91187FB542BF
ARDAXCG1187B9942B1
ARI4NXX1187FB59DF2
ARSYOWS1187FB41157
ARSVRVA1187B99228A
AR3EABF1187B98AD35
ARPTJHV11F4C83EF53
ARWUK9L11C8A42C37B
ARM0P6Z1187FB4D466
ARM7YQQ1187B9A84E7
ARKFEHQ11F50C47E43
ARYPWUK1187B98ACED
ARRDRI71187FB46EAA
ARRCDBV12454A42B90
AR9JNFU1187B9B9868
ARCGJ6U1187FB4D01F
ARL6UP41187FB5C604
AR0CWOB1187B99FA08
AR9HABI1187FB3ACAA
ARO762K1187FB46BCD
ARAOLGH12592CDA7AA
ARH9GD41187FB37E0F
ARWNTXK11EBCD7BBFB
AR87K371187FB5AA8E
ARD7U0O1187FB5531A
ARE08BN1187B98FCA3
ARYCKB61187B998E70
ARY7DVZ1187FB58433
ARCH7RE1187FB5B1B9
ARM86R71187FB4CBB4
ARZYP6O1187B9892E7
ARZ3BA51187B998A3E
ARWD25M1187FB4C563
ARB4ROA1187B9ACAEC
AR039B11187B9B30D0
ARW86ZJ11A348EFE7F
ARB90161187B98CDFE
ARN03F71187FB4E3F4
ARD4C1I1187FB4B0C3
ARXPPEY1187FB51DF4
ART9Z1P1187FB36C94
ARGFFA01187B9A2D11
ARW2Y3Q1187F

ARJB9TE1187FB3A9F8
AR0OTEX1187FB3600D
ARYJ6RP1187B9B3827
ARDUOSV1187B9894D5
ARW70HR1187B9A358C
ARQIAEA1187FB4F9F9
ARVYDAQ1187FB43338
ARPINYO12454A2E4AB
ARJ2PMY1187FB5B563
AR1PRQ21187FB3B481
ARZFRQM1187B9A9772
ARKEFM61187B9A8202
AR4GAFY1187B9B639D
ARBD4QW1187FB42153
ARB5S4L1187FB4AD20
ARBD4QW1187FB42153
ARD7U0O1187FB5531A
ARUTEVU1187B9ACAF9
AR7MNIX1187B9A2A68
AROUOZZ1187B9ABE51
ARLUF1T1187FB3603B
ARIH5GU1187FB4C958
AR02IU11187FB513F2
ARY4M3J1187B9AC94C
ARYLQVR11F50C4EFC2
AR4C7UZ1187B9AB278
ARXPPEY1187FB51DF4
ARW4ZCO1187B9B22D3
ARNEL2O1187FB4421A
ARE17W51187FB5381B
AR35YUO1187B9AD83E
AR3MSAN1187B98F06B
ARZVTZN1187FB579D4
ARHK7A51187FB411D8
ARLFUII1187FB565E5
ARYF20K1187B9B76BD
ARUW6HS1187FB3F028
AR9UPVA1187FB51260
AR1NRA91187FB404C3
ARMQHX71187B9890D3
ARTJ9GZ1187B9B46DE
ARDUBUY12086C1464B
ARKQN8X1187B99221C
ARVEJ9M1187FB4DC44
ARLI9NJ1187B9911FB
ARRR4Y81187FB524CE
ARDBFT21187B98E524
ARHKOPH11F919A768B
ARUUXNF1187B98FB36
ARBFDJW1187B9AD27A
AROPZYT11F4C83EFFA
AR0DSOO1187B9B855E
ARA2ZTN1187B

ARQXJW51187B9AEE27
ARRU9GR1187FB43F56
ARCGJ6U1187FB4D01F
ARUJ5A41187FB3F5F1
ARBMHBC11F4C8403A2
ARNG61X1187B98C7DF
ARQCHDN1187FB4C7C2
ARUJZFJ1187B9B135F
ARJ7NR71187FB437B7
AR1N6N61187FB4E25E
ARISHTQ1187FB3E6F0
ARVFGPJ1187B9A2477
ARKULSX1187FB45F84
ARHT0B11187B993CB3
AR8K7ET1187B997ABC
ARTOCWH11F50C4CE12
ARLNSD41187FB4D9A0
ARJJRZ41187FB4C856
AR3FE581187FB3F90A
ARF3L8J1187B9B7866
ARY53RR1187B9AE485
AR286CH1187B9B8441
ARO41T51187FB397AB
ARXR2N11187B99857E
AREPD8D1187FB3F5BC
AR323JD1187FB483D0
ARWLV931187B9B0F5A
ARJQ8WU1187FB37FCD
ARX9YIP1187B98A656
AR0IAWL1187B9A96D0
ARAOADX1187B98AED3
AREUYDJ1187B9BA040
AR1VCLT1187FB4C8AC
ARZDUJG1242077E2C5
AR3JLWI1187FB37141
ARR40ZV1187B9A10A8
AR7ZI7Z1187B98BF55
AREENKN1187B98F35D
ARG36WO1187B9B3A9A
AR9AM2N1187B9AD2F1
ARJDMBO1187B9AD118
AR2KSTE1187FB57A36
ARDZJ6H1187FB387F2
AREMOC71187B996C2F
AR3WMW01187B9921F7
AREPD8D1187FB3F5BC
ARVNGA71187FB3C107
ARZFRQM1187B9A9772
ARI28VJ1187FB4EAD7
ARGKD4W1187B990E04
AR07QYZ1187FB5BF78
ARV4I6F1187FB4D595
AREXX3T1187B

AR5R7791187FB3A8C3
ARHCERP1187B9912A5
ARKJVHY11E2835E76B
ARRH63Y1187FB47783
AR9YQN31187FB45B34
ARI648V1187B9B5379
ARLCGW21187B9929EE
ARW0JI11187FB592BD
ARJ2PGP119B866993D
ARCLCFS1187B9AE476
ARKD9S41187B997CA6
ARWX7AO1187B9B91AA
ARGRIHK1242077F57F
AR01IP11187B9AF5D2
AREJ5K11187B993F5F
ARODBRG1187FB3FD99
AR2SGQW1187B9B9E7C
ARW071B1187B9A603C
ARVQTZP1187FB40283
AR2N0221187FB48FB8
ARDJCM91187B9A097B
ARGSAFR1269FB35070
ARSL5SP1187B9A7AE0
AR84AQE1187B98AD9C
ARIBU2J1187FB5B09F
ARWD3OC1187FB3A666
ARM3QKQ1187FB421E0
ARF5M7Q1187FB501E8
AR00MBZ1187B9B5DB1
ARJ3TAF1187FB4AAB1
AREMPZG1187B99D246
ARGFFF91187FB4FA62
AROGWRA122988FEE45
ARMTE8E1187FB5ACC2
AR8YV0M1187B99F3E8
ARIRD6J1187FB5A98C
ARGKD4W1187B990E04
AR343TM1187B99DEFE
AR8FMEN1187FB4A657
ARLAEJO1187B98DB85
AR3UJPY1187FB5017B
ARJSGPI1241B9C8855
AR31D7E1187FB3BC62
ARISYDJ11F43A69EE1
ARE1N571187B98A47D
ARQDHPO1187B9A9F7C
ARX7R5Q1187FB5C6B3
ARTYXQC1187B9ACAB2
ARL3VGT1187FB40E8E
ARJRRJZ11F50C47E95
ARSL3N21187B98DFC5
AREWQSE1187B9AEC6C
ARLBXGO11F4C

10000

In [95]:
most_pop_aid = sorted(files_per_artist,
                      key=files_per_artist.__getitem__,
                      reverse=True)[0]
print (most_pop_aid,'has',files_per_artist[most_pop_aid],'songs.')

AROIHOI122988FEB8E has 13 songs.


In [97]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
q += " WHERE artist_id='"+most_pop_aid+"'"
res = conn.execute(q)
pop_artist_names = list(map(lambda x: x[0], res.fetchall()))
conn.close()
print ('SQL query:',q)
print ('name(s) of the most popular artist:',pop_artist_names)

SQL query: SELECT DISTINCT artist_name FROM songs WHERE artist_id='AROIHOI122988FEB8E'
name(s) of the most popular artist: ['Mario Rosenstock']


In [100]:
t1 = time.time()
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id,artist_name,Count(track_id) FROM songs"
q += " GROUP BY artist_id"
res = conn.execute(q)
pop_artists = res.fetchall()
conn.close()
t2 = time.time()
print ('found most popular artist in',strtimedelta(t1,t2))
print (sorted(pop_artists,key=lambda x:x[2],reverse=True)[0])

found most popular artist in 0:00:00.029215
('AROIHOI122988FEB8E', 'Mario Rosenstock', 13)
